In [18]:
clear

In [19]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy

tf Graph Input

In [20]:
def nn(input):

    with tf.name_scope('layer1'):
        W0 = tf.Variable(tf.truncated_normal([11, 30], stddev=0.1))
        b0 = tf.Variable(tf.constant(value=0.1, shape=[30]))
        h0 = tf.matmul([input], W0) + b0

    with tf.name_scope('layer1'):
        W1 = tf.Variable(tf.truncated_normal([30, 30], stddev=0.1))
        b1 = tf.Variable(tf.constant(value=0.1, shape=[30]))
        h1 = tf.matmul(h0, W1) + b1

    with tf.name_scope('layer1'):
        W2 = tf.Variable(tf.truncated_normal([30, 3], stddev=0.1))
        b2 = tf.Variable(tf.constant(value=0.1, shape=[3]))
        h2 = tf.matmul(h1, W2) + b2

    return h2

In [21]:
expert_policy_file = "experts/Hopper-v1.pkl"
envname = "Hopper-v1"
render = True
num_rollouts = 5

In [22]:
policy_fn = load_policy.load_policy(expert_policy_file)

('obs', (1, 11), (1, 11))


SET UP tensorflow    

In [23]:
input = tf.placeholder(tf.float32, [11])
label = tf.placeholder(tf.float32, [3])

pred = nn(input)

cost = tf.losses.absolute_difference(label, pred[0])

train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

import gym
env = gym.make(envname)
max_steps = env.spec.timestep_limit

returns = []
observations = []
actions = []

[2017-09-14 19:45:44,679] Making new env: Hopper-v1


In [26]:
 with tf.Session() as sess:
    tf_util.initialize()

    for i in range(num_rollouts):
        obs = env.reset()
        one = False
        totalr = 0.
        steps = 0
        while not done:
            action = policy_fn(obs[None,:])
            observations.append(obs)

            actions.append(action)
            obs, r, done, _ = env.step(action)
            totalr += r
            steps += 1
            if render:
                env.render()
            if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
            if steps >= max_steps:
                break
        returns.append(totalr)

    expert_data = {'observations': np.array(observations),
                   'actions': np.array(actions)}

    observations = np.array(observations)
    actions = np.array(actions)

    for k in range(30):
        #
        for i in range(observations.shape[0]):
            _, cost_, pred_action = sess.run([train_step, cost, pred], feed_dict={input: observations[i], label: actions[i][0]})

            obs = env.reset()
            done = False

        while not done:
            pred_action = sess.run(pred, feed_dict={input: obs})

            obs, r, done, _ = env.step(pred_action)
            if render:
                env.render()

IndentationError: expected an indented block (<ipython-input-26-f4a569c703e6>, line 32)